# 🍊 OPENFOODFACTS - Part 1

<ul><li>Stockage de l'ensemble des données dans une BDD</li>
<li>Connexion à votre base via un script Python</li>
<li>Répondez aux questions SQL en PJ</li></ul>

# 📄 Préparation du data pour import dans base SQL

In [3]:
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

In [24]:
path = '../data/data_raw.csv'

cols = ['product_name','quantity','nutrition-score-fr_100g','nutriscore_score', 'nutriscore_grade', 'brands', 'origins', 'ingredients_text', 'countries',
                'labels', 'energy-kcal_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'sodium_100g', 'additives', 'vitamin-c_100g', 'nova_group', 'pnns_groups_1', 'pnns_groups_2']

country = 'France'

def import_csv(path, cols):
    # load dataset
    df = pd.read_csv(path, sep='\t', error_bad_lines=False)
    # display all columns
    pd.set_option('display.max_columns', None)
    # new df with selected columns
    clean_df = df[cols]
    # replace NaN by 0 on entire dataframe
    clean_df.fillna(0)
    # replace values "en:fr" , "en:france", "en:FR" by "France" in countries
    clean_df['countries'] = clean_df['countries'].replace(['en:fr', 'en:france', 'en:FR', 'fr', 'en:France'],'France')
    # keep only the product with needed country
    clean_df = clean_df.loc[clean_df['countries'] == country]

    return clean_df


In [ ]:
df = import_csv(path, cols)

# 🔍 SQL

In [35]:
queries = ["SELECT count(*) AS 'products_count', nutriscore_grade FROM openfood.products GROUP by nutriscore_grade;",
           "SELECT nutriscore_grade, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM openfood.products) FROM openfood.products GROUP by nutriscore_grade;",
           "SELECT AVG(fiber_100g) AS 'moyenne fibres', nutriscore_grade FROM openfood.products GROUP BY nutriscore_grade;",
           "SELECT pnns_groups_2, MAX(additives_n) FROM openfood.products GROUP BY pnns_groups_2 LIMIT 10;",
           "SELECT AVG(sugars_100g) AS 'Moyenne Sucre', product_name FROM openfood.products WHERE nutriscore_grade='a' OR nutriscore_grade='b' GROUP BY product_name;",
           "SELECT COUNT(DISTINCT product_name) AS 'Total des produits' FROM openfood.products WHERE carbohydrates_100g > sugars_100g;"
           ]

def export_db(df):
    # database connection
    hostname="127.0.0.1"
    dbname="openfood"
    uname="root"
    pwd="root"

    # create SQLAlchemy engine to connect to MySQL Database
    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

    # import df into SQL db
    #df.to_sql('products', engine, index=False)

    # array containg queries df
    queries_df = []

    # export queries responses as dataframes
    for i in queries:
        queries_df.append(pd.read_sql(i, engine))

    return queries_df


In [ ]:
export_db(df)